In [1]:
pip install xgboost prophet pickle

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install dtw

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from prophet import Prophet

C:\Users\Anita Singh\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Loading the data and minor EDA

We load the data and set the index to be the DateTime column of the dataset.

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('expand_frame_repr', False)

df = pd.read_csv("../data/electricityConsumptionAndProductioction.csv")

# set the DateTime as the index
df = df.set_index("DateTime")
df.index = pd.to_datetime(df.index)

In [5]:
df_elec = pd.read_csv("../data/KOL_15mins_final_merged_numeric.csv")
df_temp=df_elec["idmeter"].unique()
print(df_temp.size)

768


In [6]:
df_batch_elec = [df_elec[df_elec["idmeter"]==x].iloc[0:104,1:] for x in df_elec["idmeter"].unique()]
df_elec_size_map = [x.size for x in df_batch_elec]
print(df_elec_size_map)

[10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088, 10088

In [14]:
from dtw import dtw
manhattan_distance = lambda x, y: np.abs(x - y)

d, cost_matrix, acc_cost_matrix, path = dtw(df_batch_elec[0].iloc[0,1:], df_batch_elec[1].iloc[0,1:], dist=manhattan_distance)
print(d)


17.44100000000001


In [33]:
n_size = len(df_elec_size_map)
m_size = df_elec_size_map[0]
distance_matrix = np.zeros((n_size,n_size))
n_size=10
for i in range(n_size):
    for j in range(n_size):
        d, cost_matrix, acc_cost_matrix, path = dtw(df_batch_elec[i].iloc[0,1:], df_batch_elec[j].iloc[0,1:], dist=manhattan_distance)
        distance_matrix[i][j] = d
        print(d)

0.0
17.44100000000001
9.932999999999991
12.295999999999998
11.588000000000005
11.076999999999991
15.984999999999994
10.604999999999997
11.949999999999998
15.275999999999996
17.44100000000001
0.0
17.942000000000004
3.2919999999999994
3.2529999999999952
8.047
42.20099999999999
5.875999999999995
9.761999999999997
1.6859999999999986
9.932999999999991
17.942000000000004
0.0
11.264
13.037999999999998
8.155999999999999
17.028
11.176999999999998
8.636000000000001
15.753
12.295999999999998
3.2919999999999994
11.264
0.0
3.530999999999996
4.8599999999999985
34.81399999999998
2.8729999999999976
7.785
1.697999999999998
11.588000000000005
3.2529999999999952
13.037999999999998
3.530999999999996
0.0
5.871999999999997
26.03899999999998
4.182999999999998
8.621999999999998
2.760999999999994
11.076999999999991
8.047
8.155999999999999
4.8599999999999985
5.871999999999997
0.0
22.798
3.413999999999999
5.265999999999999
5.945999999999998
15.984999999999994
42.20099999999999
17.028
34.81399999999998
26.0389999

Now let us print the shape fo our data set and also it's head.

In [ ]:
print(f"our shape is {df.shape}")
print()
df.head()

We attempt to predict the electricity consumption, so we will drop all the columns except the Consumption (and the DateTime which is now our index).

In [ ]:
df.drop(["Production", "Nuclear", "Wind", "Hydroelectric", "Oil and Gas", "Coal", "Solar", "Biomass"], axis=1, inplace=True)

Now let's do a plot in order to the see the data and determine if it has apttern that we can forecast.
We will plot out entiere data set.

In [ ]:
df[["Consumption"]].plot(style="-", figsize=(15, 5), title="Electricity Consumption, in MW")
plt.ylabel('MW')
plt.show()

Excelent, we can see that our entire dataste (of 4 and a half years) has a seasonality to it.

Now let us plot just a week, to see if the seasonality checks up.

In [ ]:
df["2023-01-09 00:00:00" : "2023-01-15 23:59:59"][["Consumption"]].plot(style="-", figsize=(15, 5), title="Electricity Consumption, in MW")
plt.ylabel('MW')
plt.show()

Great, we can see that our data has a very nice seasonality to it also at the week level.

# Add features and split the datain train and test sets

First let as add some new features that are derived hout the DateTime index.

In [ ]:
# method for adding time features by the time index
def createTimeFeatures(df):
    df["hour"] = df.index.hour
    df["day_of_week"] = df.index.day_of_week
    df["quarter"] = df.index.quarter
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["day_of_year"] = df.index.dayofyear


# apply the method to the existing dataframe
createTimeFeatures(df)

df.head()

Now let us split the data, we will use the last year as our dataset.

Do also a simple plot to see as a graph what is the percentage of the train versus datasets. A red vetical line will represent the split.

In [ ]:
cutOffDate = df.index[-365 * 24]
print(f"cutOffDate {cutOffDate}")

train = df.loc[df.index <= cutOffDate]
test = df.loc[df.index > cutOffDate]

print(f"train size: {len(train)} and test {len(test)}")

df[["Consumption"]].plot(style="-", figsize=(15, 5), title="Electricity Consumption, in MW")
plt.ylabel('MW')
plt.axvline(x=cutOffDate, color='r')
plt.show()

Cool, now we have augmented our dataframe with some new features and we have split the dataframe in train and test.

# Create simple method for calculating the error

We want to create a simple method that will show the error betwne the actual value and predicted value as a percentage.

In [ ]:
def meanAbsolutErrorAaPercentage(real, predicted):
    real = np.array(real)
    predicted = np.array(predicted)

    return np.mean(np.abs((real - predicted) / real)) * 100

# Forecasting using Regression

Here we will prepare our data in train and test Xs and ys and use a XGB regressor to predict the target feature or our training part of the dataset.

In [ ]:
%%time

# define our features and our target as separate arrays
FEATURES = ["hour", "day_of_week", "quarter", "month", "year", "day_of_year"]
TARGET = "Consumption"

X_train = train[FEATURES]
y_train = train[TARGET]

X_test = test[FEATURES]
y_test = test[TARGET]

# define and XGB regressor, fit it on our train data and use it to predict our test data
regressor = xgb.XGBRegressor(n_estimators=1000, early_stoppin_rounds=50, learning_rate=0.01)

regressor.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)
regressorOutput = regressor.predict(X_test)

prediction = pd.DataFrame(data=regressorOutput, index=X_test.index, columns=["prediction"])

df = df.merge(prediction, how="left", left_index=True, right_index=True)

# our predictions are at the end of the dataframe, so we should print that
df.tail()

In [ ]:
pickle.dump(regressor, open('elecModel.pkl','wb'))

Let's calculate the error withe method that we defined before.

In [ ]:
# select the part of the data frame that is our test part
testDf = df.loc[df.index > cutOffDate]

yReal = testDf["Consumption"]
yPredicted = testDf["prediction"]

print(f"percentage error: {meanAbsolutErrorAaPercentage(yReal, yPredicted):.4f}")

Great, so the percentage error between the predicted and actual value is a little over 10%, **10.1803** precisily.

Now let's plot the prediction over the actual data, first for all the dataset and after for just a week.

In [ ]:
ax = df[[TARGET]].plot(figsize=(15,5))
df["prediction"].plot(ax=ax, style=".")
ax.legend(["True Data", "Prediction"])
ax.set_title("Data and prediction")
plt.show()

In [ ]:
ax = df["2023-01-09 00:00:00" : "2023-01-15 23:59:59"][[TARGET]].plot(figsize=(15,5))
df["2023-01-09 00:00:00" : "2023-01-15 23:59:59"]["prediction"].plot(ax=ax, style=".")
ax.legend(["True Data", "Prediction"])
ax.set_title("Data and prediction")
plt.show()

So we plot the data for the entiere time range and also for just a week. The difference of about 10% is clearly visible but the regression model did pick up the pattern very well.

# Forecasting using Prophet

Prophet is a time series prediction library developed by Facebook/Meta. It is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality.

Prophet needs to be imported, as we did at the start of this notebook.

Prophets wan to have the date time column names as **ds**, and the target as **y**. So we will reset our index and rename our columns as needed.

We will do this for both our train and test dataframes.

In [ ]:
prophetTrain = train.reset_index()
prophetTrain.drop(FEATURES, axis=1, inplace=True)
prophetTrain.rename(columns={"DateTime": "ds", "Consumption": "y"}, inplace=True)

prophetTrain.tail()

In [ ]:
prophetTest = test.reset_index()
prophetTest.drop(FEATURES, axis=1, inplace=True)
prophetTest.rename(columns={"DateTime": "ds", "Consumption": "y"}, inplace=True)

prophetTest.tail()

Great, now let's create the Prophet object, fit it on the train data and use the test data for predicting.

In [ ]:
%%time

prophetModel = Prophet()
prophetModel.fit(prophetTrain)

prophetPrediction = prophetModel.predict(prophetTest)

yRealProphet = test["Consumption"]
yPredictedProphet = prophetPrediction["yhat"]

print(f"prophet percentage error: {meanAbsolutErrorAaPercentage(yRealProphet, yPredictedProphet):.4f}")

The Prophet prediction is out of the box at an about 5% error, **5.5591** precisily.

Now let's also plot the predinction for the Prophet prediction. For the entiere data set and then for just a week.

In [ ]:
# in order to be optimaly plot the data we set it's ds (our initial DateTime) as the index
prophetPrediction = prophetPrediction.set_index("ds")

ax = df[[TARGET]].plot(figsize=(15,5))
prophetPrediction["yhat"].plot(ax=ax, style=".", color="g")
ax.legend(["True Data", "Prediction"])
ax.set_title("Data and prediction")
plt.show()

In [ ]:
ax = df["2023-01-09 00:00:00" : "2023-01-15 23:59:59"][[TARGET]].plot(figsize=(15,5))
prophetPrediction["2023-01-09 00:00:00" : "2023-01-15 23:59:59"]["yhat"].plot(ax=ax, style=".", color="g")
ax.legend(["True Data", "Prediction"])
ax.set_title("Data and prediction")
plt.show()

So the Prophet prediction looks pretty good, after plotting it's easy to see that the trend is followed pretty well and we get a smaller error.